# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [16]:
import ames_fe # The module for loading external data - Ames Housing dataset
import os
import pandas as pd
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-152755
aml-quickstarts-152755
southcentralus
a0a76bad-11a1-4a2d-9887-97a29122c8ed


In [3]:
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [17]:
train, test = ames_fe.load_data_tailored()
print(f"train.shape = {train.shape}, test.shape = {test.shape}")

train.shape = (1460, 78), test.shape = (1459, 78)


In [18]:
# Register dataset
blob = ws.get_default_datastore()
train_reg = TabularDatasetFactory.register_pandas_dataframe(train, blob, name='Ames-housing-dataset',
                                                          description='Ames Housing training data.')

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/070139d9-b7de-4d14-86fc-30a1fcb87b8b/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [19]:
# Choose a name for experiment
experiment_name = 'Ames-housing-AutoML'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [23]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_settings = {
    'experiment_timeout_minutes' : 120,
    'n_cross_validations' : 3,
    'enable_early_stopping' : True,
    'iteration_timeout_minutes' : 5,
    'max_concurrent_iterations' : 4,
    'max_cores_per_iteration' : -1
}

automl_config = AutoMLConfig(
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    compute_target=cpu_cluster,
    training_data=train_reg,
    label_column_name='SalePrice',
    **automl_settings)

In [ ]:
from azureml.widgets import RunDetails
# TODO: Submit your experiment
aml_run = experiment.submit(automl_config)
RunDetails(aml_run).show()
aml_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get your best run and save the model from that run.
best_run, fitted_model = aml_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Run(Experiment: Ames-housing-AutoML,
Id: AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
)), ('elasticnet', ElasticNet(alpha=0.05357894736842105, copy_X=True, fit_intercept=True, l1_ratio=0.6352631578947369, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection='cyclic', tol=0.0001, warm_start=False))], verbose=False))], weights=[0.4, 0.13333333333333333, 0.26666666666666666, 0.06666666666666667, 0.13333333333333333]))],
                                     verbo

In [9]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'r2_score': 0.885898472424608,
 'normalized_root_mean_squared_log_error': 0.042947817229549566,
 'mean_absolute_error': 15439.116472115917,
 'normalized_root_mean_squared_error': 0.037139291460024836,
 'spearman_correlation': 0.9573852130501542,
 'median_absolute_error': 9654.516791774067,
 'root_mean_squared_error': 26744.00378036388,
 'mean_absolute_percentage_error': 9.122770171936464,
 'normalized_mean_absolute_error': 0.021440239511339974,
 'explained_variance': 0.886052769069071,
 'root_mean_squared_log_error': 0.13203033398097183,
 'normalized_median_absolute_error': 0.013407188990104244,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36/predicted_true'}

In [10]:
import json
with open('best_run_metrics.json', 'w') as file:
    json.dump(best_run_metrics, file)
with open('best_run_details.txt', 'w') as file:
    file.write(str(best_run.get_details()))

In [13]:
# Test the best model accuracy in the Kaggle competition
test = pd.read_csv('test.csv', index_col='Id')
test.shape

(1459, 79)

In [26]:
# NaN values not treated by the model .... 
y_hat = fitted_model.predict(test.fillna(0))
y_hat[:10]

array([124776.48048299, 160882.70997397, 186544.04940138, 189789.33383424,
       188952.25610385, 173681.83574201, 172458.43604907, 163224.08993769,
       187574.6388713 , 132248.48694095])

In [27]:
len(y_hat)

1459

In [33]:
with open('automl-submission.csv', 'w') as file:
    pd.DataFrame({'Id' : test.index.to_list(), 'SalePrice': y_hat}).to_csv(file, index=False)

In [34]:
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.config',
 '.ipynb_checkpoints',
 'automl-submission.csv',
 'automl.ipynb',
 'automl.log',
 'azureml_automl.log',
 'best_run_details.txt',
 'best_run_metrics.json',
 'data_description.txt',
 'house-prices-advanced-regression-techniques.zip',
 'Register_dataset.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [36]:
# Set the enviroment variables
os.environ['KAGGLE_USERNAME'] = "lubomrstraka"
os.environ['KAGGLE_KEY'] = "c7347462ef834e6645ce238c2f2fa561"
! kaggle competitions submit -c house-prices-advanced-regression-techniques -f automl-submission.csv -m "Submission generated by Azure Automated ML."

100%|██████████████████████████████████████| 33.7k/33.7k [00:01<00:00, 27.0kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

> 2021-07-23: Your submission scored 0.13612, which is not an improvement of your best score (0.12481).

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
# Check the path to the model
best_run.get_file_names()[-6] # where is output/model.joblib ?[-4]? CHECK it in the next cell

'outputs/model.pkl'

In [3]:
# Save the best model
from azureml.core import Model
os.makedirs('./outputs/', exist_ok=True)
best_run.download_file(best_run.get_file_names()[-6], output_file_path='./outputs/')
model = Model.register(ws, model_path='outputs/model.pkl', model_name='Ames-Housing-AutoML-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

NameError: name 'best_run_metrics' is not defined

In [4]:
# Retrive previously saved the best model
from azureml.core import Model
os.makedirs('./outputs/', exist_ok=True)
model = Model.register(ws, model_path='outputs/model.pkl', model_name='Ames-Housing-AutoML-Model')
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-AutoML-Model
Ames-Housing-AutoML-Model	Ames-Housing-AutoML-Model:2	2


In [5]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "AutoML"},
                                              description="Predict Ames Housing Prices")

In [9]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('conda_env_v_1_0_0.yml')
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-AutoML-Model')

myenv = Environment.get(workspace=ws, name="project-env", version="1")
inference_config = InferenceConfig(entry_script="scoring_file_v_1_0_0.py", environment=myenv)

service_name = 'ames-housing-aml-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-23 11:15:59+00:00 Creating Container Registry if not exists..
2021-07-23 11:16:10+00:00 Registering the environment..
2021-07-23 11:16:11+00:00 Building image.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()